In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from grok import Grok
import pickle

In [5]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
print(grok_api_key, grok_url)
grok = Grok(grok_api_key, grok_url, grok_model)

xai-fq2OPXVKJrQCMyitw8z4fUiuVE6OO0KBjy1AzT5L56DiZsG88SqNJgvISwXC7psVI6wmT5vkAkFWadXF https://api.x.ai/v1/chat/completions


In [6]:
def generate_script_grok(client: Grok, trend_data: dict, news_report: str) -> str:

    """Generate a video script using Grok based on trend data"""
    
    prompt = f"""Create a very short, 20-second video script about this trending topic:
    Topic: {trend_data['keyword']}
    Related Keywords: {', '.join(trend_data['keywords'])}
    News Report: {news_report}
    
    The script should be:
    1. Informative and accurate
    2. Focused on the most important and sensational aspects of the story
    3. Include important big-picture context
    4. Conversational and concise.
    5. At most 20 seconds in length.
    6. Less than 80 words in length.

    The script should NOT:
    1. Contain an outro of any kind, or mention subscribing to the channel
    2. Use acronyms or abbreviations without explanation"""

    return client.generate_text(prompt)

In [7]:
def generate_sora_prompts_grok(client: Grok, script: str, num_prompts: int = 1) -> list[str]:
    """Generate Sora-compatible video prompts based on the script"""

    soraScripts = []
    prompt = f"""Break down this script into a series of four video scene descriptions suitable for AI video generation:
    
    Script: {script}
    
    There should be four scenes.

    For each scene:
    
    1. Describe the visual elements in detail
    2. Include camera movements and transitions
    3. Be concise -- under 25 words.
    4. Use loose, conversational language in your description.
    5. Format the scene description as follows:
        [Scene Title] <= 5 words
        [Scene Description] <= 25 words
        [Camera Movements/Transitions] <= 25 words
        newline

    Do NOT:
    1. Generate a scene for an outro
    2. Use acronyms or abbreviations without explanation
    """
    
    for i in range(num_prompts):
        soraScript = client.generate_text(prompt)
        soraScripts.append(soraScript)
    return soraScripts

In [37]:
def generate_grok_image_prompts(client: Grok, script: str, num_prompts: int = 10) -> list[str]:
    """Generate Grok image prompts based on the script"""

    prompt = f"""Generate a series of (~{int}) image descriptions that capture the most sensational themes of the following script.

    For each image:
    1. Be assertive and highly creative in your description. 
    2. Be clear and concise. 
    3. Use familiar elements of popular culture where appropriate e.g. brands, celebrities, landmarks, products, etc.

    Each image prompt should have the following format:
    Generate an image of [description]

    Separate each image prompt with a newline.
    
    Script: {script}
    """
    return client.generate_text(prompt)

In [9]:
def generate_grok_narration(client: Grok, script: str) -> str:
    """Generate a narration using Grok based on the script"""

    prompt = f"""Extract only the words to be spoken from this video script:
    Script: {script}.

    The result should NOT include any mention of a narrator or scene descriptions.
    """
    return grok.generate_text(prompt)

In [10]:
def generate_hashtags(client: Grok, script: str) -> list[str]:
    """Generate hashtags using Grok based on the script"""

    prompt = f"""Generate the best series of hashtags to make this video go viral on instagram reels, tik tok, and youtube shorts.
    Select the best 10 hashtags and brief reasoning for each.

    The result should be in the following format:
    for each hashtag:
        [Hashtag] [Reasoning]

    Include a one-line space separated list of all hashtags at the end.

    Script: {script}"""
    return client.generate_text(prompt)


In [11]:
grok_api_key, grok_url, grok_model = os.getenv("GROK_API_KEY"), os.getenv("GROK_URL"), 'grok-2-latest'
grok = Grok(grok_api_key, grok_url, grok_model)

In [12]:
# List directories in assets and get most recent
asset_dirs = [d for d in os.listdir('assets') if os.path.isdir(os.path.join('assets', d))]
RUN_DIR = os.path.join('assets', max(asset_dirs))
MAIN = [os.path.join(RUN_DIR, d) for d in os.listdir(RUN_DIR) if os.path.isdir(os.path.join(RUN_DIR, d))][0] # only one directory in the RUN_DIR
NEWS_REPORT = os.path.join(MAIN, 'news_report.txt')
SORAPROMPTS = os.path.join(MAIN, 'soraprompts.txt')
SCRIPT = os.path.join(MAIN, 'script.txt')
keyword = MAIN.split('\\')[-1].replace('_', ' ')

In [13]:
finalTrendList = pickle.load(open('./tmp/finalTrendList.pkl', 'rb'))

In [14]:
trend = [trend for trend in finalTrendList if trend['keyword'] == keyword][0]

In [15]:
with open(NEWS_REPORT) as f:
    newsReport = f.read()
print(newsReport)

**Mass Layoffs at U.S. Department of Education Marks a New Chapter Under Trump Administration**

*Date: March 12, 2025 | By: [Your News Agency Name]*

In a controversial move that has sent shockwaves across the educational landscape, the U.S. Department of Education announced on March 11 that it will lay off nearly 1,300 employees, effectively halving its workforce as part of an effort to eliminate what it calls "bureaucratic bloat." This massive staff reduction comes less than two months into President Donald Trump's second term, fulfilling a long-standing campaign promise to dismantle the agency, which he deems to be ineffective and excessively bureaucratic.

**Significant Downsizing**

Education Secretary Linda McMahon, the newly installed chief of the agency and former CEO of World Wrestling Entertainment, stated that this "reduction in force" (RIF) aims to bolster efficiency and accountability within the department. Following these layoffs, the department's staff will shrink from 

In [21]:
GROKPATH = MAIN + '/grok'
os.mkdir(GROKPATH)

In [22]:
soraPromptPath = GROKPATH + '/grok-soraprompts.txt'
scriptPath = GROKPATH + '/grok-script.txt'
grokImagePromptsPath = GROKPATH + '/grok-imageprompts.txt'
grokNarrationPath = GROKPATH + '/grok-narration.txt'
grokHashtagsPath = GROKPATH + '/grok-hashtags.txt'
opinionAnalysisPath = GROKPATH + '/grok-opinionanalysis.txt'

In [23]:
script = generate_script_grok(grok, trend, newsReport)
with open(scriptPath, 'w') as f:
    f.write(script)
print(script)

**Scene: Newsroom setting with a news anchor**

Anchor: "Breaking news: The U.S. Department of Education, led by Secretary Linda McMahon, has announced massive layoffs, cutting its workforce in half. This move, part of President Trump's second-term agenda, aims to reduce what they call 'bureaucratic bloat.' Critics fear this will harm students, especially those from disadvantaged backgrounds, as key services like federal student loans and civil rights enforcement are at risk."


In [38]:
images = generate_grok_image_prompts(grok, script)
with open(grokImagePromptsPath, 'w') as f:
    f.write(images)
print(images)

Generate an image of a dramatic newsroom scene where a stern-faced news anchor, reminiscent of Anderson Cooper, delivers the breaking news. Behind him, a large screen displays the U.S. Department of Education logo being sliced in half by a giant pair of scissors, symbolizing the massive layoffs.

Generate an image of Linda McMahon, styled like a WWE character, standing triumphantly atop a pile of discarded office furniture and shredded documents, representing the reduction of 'bureaucratic bloat' in the Department of Education.

Generate an image of a distressed student, looking like a young Emma Watson, surrounded by fading images of school books, a graduation cap, and a 'Federal Student Loans' sign, illustrating the potential harm to students from disadvantaged backgrounds.

Generate an image of President Trump, in a superhero costume labeled 'Bureaucracy Buster', using a giant hammer to smash through a wall labeled 'Department of Education', emphasizing his second-term agenda.

Gene

In [25]:
soraPrompts = generate_sora_prompts_grok(grok, script)
with open(soraPromptPath, 'w') as f:
    f.write(soraPrompts[0])

In [26]:
print(soraPrompts[0])

[Newsroom Setup]
Anchor at desk, serious look, newsroom background with screens.
Camera zooms in on anchor, steady shot.

[Announcement of Layoffs]
Anchor speaks, graphic shows U.S. Department of Education logo.
Cut to graphic, then back to anchor, smooth transition.

[Impact on Students]
Anchor's expression turns concerned, text overlay about student services.
Camera pans to text overlay, then back to anchor, slow zoom out.

[Political Context]
Anchor mentions President Trump, image of Trump appears briefly.
Quick cut to Trump's image, then dissolve back to anchor.


In [27]:
grokNarration = generate_grok_narration(grok, script)
with open(grokNarrationPath, 'w') as f:
    f.write(grokNarration)
print(grokNarration)
print("WORD COUNT: ", len(grokNarration.split(' ')))


"Breaking news: The U.S. Department of Education, led by Secretary Linda McMahon, has announced massive layoffs, cutting its workforce in half. This move, part of President Trump's second-term agenda, aims to reduce what they call 'bureaucratic bloat.' Critics fear this will harm students, especially those from disadvantaged backgrounds, as key services like federal student loans and civil rights enforcement are at risk."
WORD COUNT:  62


In [28]:
hashtags = generate_hashtags(grok, script)
with open(grokHashtagsPath, 'w') as f:
    f.write(hashtags)
print(hashtags)


1. **#BreakingNews** - This hashtag is essential as it directly relates to the format and urgency of the content, attracting viewers interested in current events.
2. **#EducationCuts** - Specific to the video's topic, this hashtag will draw in viewers concerned about educational policy and funding.
3. **#TrumpAgenda** - Relevant to the political context provided in the script, this will attract viewers interested in Trump's policies.
4. **#LindaMcMahon** - Mentioning the Education Secretary by name in a hashtag can pull in viewers who follow political figures or are interested in her actions.
5. **#BureaucraticBloat** - This phrase is used in the script and can attract viewers interested in government efficiency and reform.
6. **#StudentLoans** - A key issue mentioned in the script, this hashtag will attract viewers concerned about student financial aid.
7. **#CivilRights** - Another critical issue mentioned, this hashtag will draw viewers interested in civil rights and educational equ

In [29]:
def generate_grok_opinion_analysis(client: Grok, news_report: str) -> str:
    """Generate an opinion analysis using Grok based on the news report"""

    prompt = f"""Generate an opinion analysis using Grok based on the news report:
    News Report: {news_report}

    The opinion analysis should:
    1. Be highly engaging and bold.
    2. Provide evidence based reasoning for claims made. 
    3. Be less than 500 words in length.
    4. Be written in a professional tone.

    The opinion analysis should NOT:
    1. Cite sources.

    Format:
    [Title Highlighting the Central Claim] <= 10 words
    [Opinion Analysis] <= 500 words
    """

    return client.generate_text(prompt)

In [30]:
opinionAnalysis = generate_grok_opinion_analysis(grok, newsReport)

In [31]:
with open(opinionAnalysisPath, 'w') as f:
    f.write(opinionAnalysis)
print(opinionAnalysis)


**Trump's Education Cuts: A Reckless Assault on America's Future**

The Trump administration's decision to gut the U.S. Department of Education, slashing its workforce by nearly half, is a reckless and shortsighted move that threatens the very foundation of our nation's educational system. This drastic reduction in personnel, justified by baseless claims of "bureaucratic bloat," will inevitably lead to diminished services and increased hardships for the most vulnerable students.

The loss of over 1,300 employees, including those responsible for managing federal student loans, overseeing grants, and enforcing civil rights laws, will severely impair the department's ability to fulfill its critical mission. The impact will be felt most acutely by students with disabilities, those from low-income backgrounds, and other marginalized groups who rely on these essential programs for access to education and opportunity.

Proponents of these cuts argue that shifting control to state governments 

In [32]:
len(grokNarration.split(' '))

62